In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pandas as pd
from datetime import datetime
import os
import time

In [ ]:
old_file = '.\\wulpus\\measurements\\fliessfront-2025-08-29.npz'
recording = time.strptime("2025-08-29 14:00", "%Y-%m-%d %H:%M")

data = np.load(old_file)
path = os.path.dirname(old_file)
print('Keys:', data.files)

In [ ]:
data_cnt = data['data_arr'].shape[1]
num_samples = data['data_arr'].shape[0]
measurements = data['data_arr'].T

tx_channel = np.zeros((num_samples,1), dtype=np.uint8)
rx_channel = np.zeros((num_samples,1), dtype=np.uint8)
time = np.zeros(num_samples, dtype=np.float32)

config = [
    [[1, 2, 3, 4, 5, 6, 7], [0]],
    [[0, 2, 3, 4, 5, 6, 7], [1]],
    [[0, 1, 3, 4, 5, 6, 7], [2]],
    [[0, 1, 2, 4, 5, 6, 7], [3]],
    [[0, 1, 2, 3, 5, 6, 7], [4]],
    [[0, 1, 2, 3, 4, 6, 7], [5]],
    [[0, 1, 2, 3, 4, 5, 7], [6]],
    [[0, 1, 2, 3, 4, 5, 6], [7]],
]

In [ ]:
recording_dt = datetime(*recording[:6])
dates = pd.date_range(recording_dt, periods=num_samples, freq="s")
df = pd.DataFrame({
        'measurement': [pd.Series(measurements[i]) for i in range(num_samples)], 
        "tx": [config[i % len(config)][0] for i in range(num_samples)], 
        "rx": [config[i % len(config)][1] for i in range(num_samples)],
        "aq_number": [i for i in range(num_samples)],
        "log_version": 1,
        "tx_rx_id": [i % len(config) for i in range(num_samples)]
    }, 
    index=dates)
df.head(15)

In [ ]:
# Expand measurement series into columns so they're all included in save-format (parquet)
import io
from zipfile import ZipFile

measurement_expanded = pd.DataFrame(
    [m.values for m in df['measurement']], index=df.index)
flattened_df = pd.concat(
    [df.drop(columns=['measurement']), measurement_expanded], axis=1)
flattened_df.columns = [str(col) for col in flattened_df.columns]

with ZipFile("converted.zip", 'w') as zf:
    # zf.writestr('config-0.json', config_json.dumps())
    # Write dataframe as parquet
    buffer = io.BytesIO()
    flattened_df.to_parquet(buffer)
    zf.writestr('data.parquet', buffer.getvalue())